# Direct beam iterations for LoKI

## Introduction

This notebook is used to compute the direct beam function for the LoKI detectors.
It uses data recorded during the detector test at the Larmor instrument.

**Description of the procedure:**

The idea behind the direct beam iterations is to determine an efficiency of the detectors as a function of wavelength.
To calculate this, it is possible to compute $I(Q)$ for the full wavelength range, and for individual slices (bands) of the wavelength range.
If the direct beam function used in the $I(Q)$ computation is correct, then $I(Q)$ curves for the full wavelength range and inside the bands should overlap.

In the following notebook, we will:

1. Create a pipeline to compute $I(Q)$ inside a set of wavelength bands (the number of wavelength bands will be the number of data points in the final direct beam function)
1. Create a flat direct beam function, as a function of wavelength, with wavelength bins corresponding to the wavelength bands
1. Calculate inside each band by how much one would have to multiply the final $I(Q)$ so that the curve would overlap with the full-range curve
   (we compute the full-range data by making a copy of the pipeline but setting only a single wavelength band that contains all wavelengths)
1. Multiply the direct beam values inside each wavelength band by this factor
1. Compare the full-range $I(Q)$ to a theoretical reference and add the corresponding additional scaling to the direct beam function
1. Iterate until the changes to the direct beam function become small

In [ ]:
import numpy as np
import scipp as sc
import sciline
import scippneutron as scn
import plopp as pp
from ess import sans
from ess import loki
from ess import isissans as isis
from ess.sans.types import *

## Define reduction parameters

We define a dictionary containing the reduction parameters, with keys and types given by aliases or types defined in `ess.sans.types`:

In [ ]:
params = loki.default_parameters.copy()

# File names
params[Filename[SampleRun]] = '60339-2022-02-28_2215.nxs'
params[Filename[BackgroundRun]] = '60393-2022-02-28_2215.nxs'
params[Filename[TransmissionRun[SampleRun]]] = '60394-2022-02-28_2215.nxs'
params[Filename[TransmissionRun[BackgroundRun]]] = '60392-2022-02-28_2215.nxs'
params[Filename[EmptyBeamRun]] = '60392-2022-02-28_2215.nxs'

# Wavelength binning parameters
wavelength_min = sc.scalar(1.0, unit='angstrom')
wavelength_max = sc.scalar(13.0, unit='angstrom')
n_wavelength_bins = 50
n_wavelength_bands = 50

params[WavelengthBins] = sc.linspace(
    'wavelength', wavelength_min, wavelength_max, n_wavelength_bins + 1
)
params[WavelengthBands] = sc.linspace(
    'wavelength', wavelength_min, wavelength_max, n_wavelength_bands + 1
)

masks = ['mask_new_July2022.xml']
banks = ['larmor_detector']

params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
params[ReturnEvents] = False

params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.3, num=101, unit='1/angstrom')

## Create pipeline using Sciline

We use all providers available in `esssans` as well as the `loki`-specific providers,
which include I/O and mask setup specific to the [LoKI](https://europeanspallationsource.se/instruments/loki) instrument.

We then build the pipeline which can be used to compute the (background subtracted) $I(Q)$.

In [ ]:
providers = sans.providers + loki.providers + (isis.io.read_xml_detector_masking,)

pipeline = sciline.Pipeline(providers, params=params)
pipeline.insert(sans.merge_banks)
pipeline.set_param_series(PixelMaskFilename, masks)
pipeline.set_param_series(NeXusDetectorName, banks)

# Add providers that fetch data from online resource
for provider in loki.data.providers:
    pipeline.insert(provider)

# In the present file, there is no sample information so we use a dummy sample provider
pipeline.insert(loki.io.dummy_load_sample)

Before we begin computations, we can visualize the pipeline:

In [ ]:
pipeline.visualize(BackgroundSubtractedIofQ, compact=True, graph_attr={'rankdir': 'LR'})

## Finding the beam center

Looking carefully at the pipeline above,
one will notice that there is a missing parameter from the pipeline: the red box that contains the `BeamCenter` type.
Before we can proceed with computing the direct beam function,
we therefore have to first determine the center of the beam.

There are more details on how this is done in the [beam center finder notebook](../common/beam-center-finder.ipynb),
but for now we simply reuse the pipeline (by making a copy),
and inserting the provider that will compute the beam center.

For now, we compute the beam center only for the rear detector (named 'larmor_detector') but apply it to all banks (currently there is only one bank).
The beam center may need to be computed or applied differently to each bank, see [scipp/esssans#28](https://github.com/scipp/esssans/issues/28).

In [ ]:
bc_pipeline = pipeline.copy()
bc_pipeline.del_param_table(NeXusDetectorName)
bc_pipeline[NeXusDetectorName] = 'larmor_detector'
bc_pipeline.insert(sans.beam_center_from_center_of_mass)

In [ ]:
bc_pipeline.visualize(BeamCenter, compact=True, graph_attr={'rankdir': 'LR'})

We can now compute the value for the center

In [ ]:
center = bc_pipeline.compute(BeamCenter)
center

and set that value onto our original pipeline

In [ ]:
pipeline[BeamCenter] = center

## Expected intensity at zero Q

The sample used in the experiment has a known $I(Q)$ profile,
and we need it to calibrate the absolute intensity of our $I(Q)$ results
(relative differences between wavelength band and full-range results are not sufficient).

We load this theoretical reference curve, and compute the $I_{0}$ intensity at the lower $Q$ bound of the range covered by the instrument.

In [ ]:
from scipp.scipy.interpolate import interp1d
from ess.loki.data import get_path

Iq_theory = sc.io.load_hdf5(get_path('PolyGauss_I0-50_Rg-60.h5'))
f = interp1d(Iq_theory, 'Q')
I0 = f(sc.midpoints(params[QBins])).data[0]
I0

## A single direct beam function for all layers

As a first pass, we compute a single direct beam function for all the detector pixels combined.

We compute the $I(Q)$ inside the wavelength bands and the full wavelength range,
derive a direct beam factor per wavelength band,
and also add absolute scaling using the reference $I_{0}$ value

In [ ]:
results = sans.direct_beam(pipeline=pipeline, I0=I0, niter=6)
# Unpack the final result
iofq_full = results[-1]['iofq_full']
iofq_bands = results[-1]['iofq_bands']
direct_beam_function = results[-1]['direct_beam']

We now compare the $I(Q)$ curves in each wavelength band to the one for the full wavelength range (black).

In [ ]:
pp.plot(
    {**sc.collapse(iofq_bands, keep='Q'), **{'full': iofq_full}},
    norm='log',
    color={'full': 'k'},
    legend=False,
)

The overlap is satisfactory, and we can now inspect the direct beam function we have computed:

In [ ]:
direct_beam_function.plot(vmax=1)

Finally, as a sanity check, we compare our final $I(Q)$ for the full wavelength range to the theoretical reference:

In [ ]:
pp.plot(
    {'reference': Iq_theory, 'data': iofq_full},
    color={'reference': 'darkgrey', 'data': 'C0'},
    norm='log',
)

## Direct beam function per layer

The LoKI detector tubes are arranged in layers along the beam path,
where the layers closest to the sample will receive most of the scattered neutrons,
while occulting the layers behind them.

A refinement to the above procedure is to compute a direct beam function for each layer of tubes individually.
We also use the 4 thick layers of tubes, but in principle,
this could also be done for 28 different layers (made from the `layer` and `straw` dimensions) if a run with enough events is provided (or many runs are combined together).

The only other difference compared to the computation above is that we now want our final result to preserve the `'layer'` dimension,
so that the dimensions of our result are `['layer', 'Q']`.

In [ ]:
pipeline[DimsToKeep] = ['layer']

Now we are able to run the direct-beam iterations on a per-layer basis:

In [ ]:
results_layers = sans.direct_beam(pipeline=pipeline, I0=I0, niter=6)
# Unpack the final result
iofq_full_layers = results_layers[-1]['iofq_full']
iofq_bands_layers = results_layers[-1]['iofq_bands']
direct_beam_function_layers = results_layers[-1]['direct_beam']

We can now inspect the wavelength slices for the 4 layers.

In [ ]:
plots = [
    pp.plot(
        {
            **sc.collapse(iofq_bands_layers['layer', i], keep='Q'),
            **{'full': iofq_full_layers['layer', i]},
        },
        norm='log',
        color={'full': 'k'},
        legend=False,
        title=f'Layer {i}',
    )
    for i in range(4)
]

(plots[0] + plots[1]) / (plots[2] + plots[3])

Now the direct beam function inside each layer looks like:

In [ ]:
pp.plot(sc.collapse(direct_beam_function_layers, keep='wavelength'))

And finally, for completeness, we compare the $I(Q)$ to the theoretical reference inside each layer.

In [ ]:
layers = sc.collapse(iofq_full_layers, keep='Q')
pp.plot(
    {**{'reference': Iq_theory}, **layers},
    color={
        **{'reference': 'darkgrey'},
        **{key: f'C{i}' for i, key in enumerate(layers)},
    },
    norm='log',
)

## Combining multiple runs to boost signal

It is common practise to combine the events from multiple runs to improve the statistics on the computed $I(Q)$,
and thus obtain a more robust direct beam function.

To achieve this, we need to replace the `SampleRun` and `BackgroundRun` file names with parameter series.
We then need to supply additional providers which will merge the events from the runs appropriately
(note that these providers will merge both the detector and the monitor events).

We first define a list of file names for the sample and background runs (two files for each):

In [ ]:
del params[Filename[SampleRun]]
del params[Filename[BackgroundRun]]

sample_runs = ['60250-2022-02-28_2215.nxs', '60339-2022-02-28_2215.nxs']
background_runs = ['60248-2022-02-28_2215.nxs', '60393-2022-02-28_2215.nxs']

We now construct a new pipeline, inserting parameter series and merging providers:

In [ ]:
# Same as original pipeline, but without sample and background run file names
params[BeamCenter] = center
pipeline = sciline.Pipeline(providers, params=params)
for provider in loki.data.providers:
    pipeline.insert(provider)
pipeline.insert(sans.merge_banks)
pipeline.set_param_series(PixelMaskFilename, masks)
pipeline.set_param_series(NeXusDetectorName, banks)

# Set parameter series for file names
pipeline.set_param_series(Filename[SampleRun], sample_runs)
pipeline.set_param_series(Filename[BackgroundRun], background_runs)

# Add event merging provider
pipeline.insert(sans.merge_runs)

# Add dummy sample provider
pipeline.insert(loki.io.dummy_load_sample)

If we now visualize the pipeline,
we can see that every step for the `SampleRun` and `BackgroundRun` branches are now series of types (3D-looking boxes instead of flat rectangles).
There is also the new `merge_multiple_runs` step that combines the events from the two runs,
just before the normalization.

In [ ]:
pipeline.visualize(BackgroundSubtractedIofQ, compact=True, graph_attr={'rankdir': 'LR'})

We run the direct beam iterations again and compare with our original results.

In [ ]:
results = sans.direct_beam(pipeline=pipeline, I0=I0, niter=6)
# Unpack the final result
iofq_full_new = results[-1]['iofq_full']
iofq_bands_new = results[-1]['iofq_bands']
direct_beam_function_new = results[-1]['direct_beam']

In [ ]:
pp.plot({'one run': direct_beam_function, 'two runs': direct_beam_function_new})

In [ ]:
pp.plot(
    {'reference': Iq_theory, 'one run': iofq_full, 'two runs': iofq_full_new},
    color={'reference': 'darkgrey', 'one run': 'C0', 'two runs': 'C1'},
    norm='log',
)